In [17]:
# DESCRIÇÃO DO EXERCÍCIO 05 - 02

# Aluno: Wekler Mendes Sarmento

# Para fazer esta atividade você precisará assistir o vídeo https://youtu.be/j8-dT-OoYFs.

# Depois você precisará submeter uma predição no Kaggle e salvar a página de submissão em PDF.

# Salve o PDF no seu GitHub e coloque o link para entregar a atividade.

# A sua nota será o score multiplicado por 100.

# Veja o exemplo da minha submissão: 
# https://github.com/fboldt/aulasml/blob/master/Titanic%20-%20Machine%20Learning%20from%20Disaster%20_%20Kaggle.pdf

# Minha nota com essa submissão seria 76.

###############################
# DEPENDÊNCIAS
###############################
import pandas as pd # Para a leitura dos dados em arquivo
import numpy as np # Para os cálculos aritiméticos
from sklearn.metrics import accuracy_score # Para cálculo da precisao SE NECESSÁRIO
from sklearn.model_selection import cross_validate # Calculando a precisão com o cross_validate
from sklearn.model_selection import GridSearchCV # Calculando a precisão utilizando o melhor parâmetro (GridSearch)
from sklearn.tree import DecisionTreeClassifier # Usando um classificador em árvore
from sklearn.pipeline import FeatureUnion # Para fazer a união dos dados após tratamento
from sklearn.pipeline import Pipeline # Para trabalhar com pipelines
from sklearn.impute import SimpleImputer # Substitui os valores nulos de um conjunto baseado em uma condição
from sklearn.preprocessing import StandardScaler # Para fazer a padronização dos dados
from sklearn.preprocessing import OneHotEncoder # Utilizando o Encoder
from sklearn.base import BaseEstimator, TransformerMixin # Utilizar na criação da classe (necessário para o cross validate)

# # Testando
# import os
# import random
# from sklearn.linear_model import LogisticRegression

################################################################
# Importando os dados da planilha de treino e separando em X e y
################################################################

data = pd.read_csv('train.csv') # Arquivo para treino do Titanic
y_train = data['Survived']
X_train = data.drop('Survived',axis=1) # Coloca todas as colunas no X menos o que já foi colocado em y (Survived)


###############################################################
# Importando os dados da planilha de teste e separando em X e y
###############################################################

X_test = pd.read_csv('test.csv') # Arquivo para teste do Titanic
#X_test = data.drop('Survived',axis=1) # Coloca todas as colunas no X menos o que já foi colocado em y (Survived)


# Criando uma classe que separa somente as colunas categóricas
class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns # Seleciona de X somente as colunas com tipo categórico (object)
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas]

# Criando uma classe que separa somente as colunas numéricas
class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns # Seleciona de X somente as colunas com tipo numérico
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas]

# Criando uma classe que separa somente as colunas desejadas
class AtributosDesejados(BaseEstimator, TransformerMixin): # Classe criada para retirar as colunas indesejadas de forma automática
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Name', 'Ticket', 'Cabin']
        return self
    def transform(self, X, y=None):
        return X.drop(self.colunasIndesejadas,axis=1)

pipenum = Pipeline([ # Criando um pipeline para automatizar as tarefas dos valores numéricos em um conjunto de dados
    ('atributos_numericos', AtributosNumericos()), # Seleciona somente os valores numéricos
    ('imputer', SimpleImputer(strategy='median')), # Substitui os valores nulos com a mediana dos demais valores
    ('scaler', StandardScaler()) # Fazendo a padronização
])

pipecat = Pipeline([ # Criando um pipeline para automatizar as tarefas dos valores categóricos em um conjunto de dados
    ('atributos_categoricos', AtributosCategoricos()), # Seleciona somente os valores categóricos
    ('imputer', SimpleImputer(strategy='most_frequent')), # Substitui os valores nulos com os valores mais frequentes
    ('encoder', OneHotEncoder()) # Colocando o encoder
])

unecaracteristicas = FeatureUnion([ # um pipeline para juntas tanto as características numéricas tratadas quanto as categóricas tratadas
    ('pipenum', pipenum), # Reutiliza os pipelines criados anteriormente
    ('pipecat', pipecat)
])

preproc = Pipeline([ # Agora o objetivo é receber um X quadrado e já sair totalmente redondo e pronto para usar
    ('atributos_desejados', AtributosDesejados()),
    ('unecaracteristicas', unecaracteristicas)
])

# Alterado tomando como base o teste do site: https://www.kaggle.com/ihelon/titanic-hyperparameter-tuning-with-gridsearchcv
pipetotal = Pipeline([ # A intenção é fazer o pipeline geral dado os dados não tratados ele já faz a predição
    ('preproc', preproc),
    ('arvore', DecisionTreeClassifier())
])

#pipetotal.get_params().keys()

parametros = { # Parametros a serem utilizados no GridSearch
    'arvore__max_depth': [None] + list(range(1,20,2)), # A profundidade máxima da árvore é por padrão vazio (None), mas vai de 1 à 20 de 2 em 2 (1,20,2)
    'arvore__min_samples_split': list(range(2,100)), # Quantidade mínima de amostras
    'arvore__splitter':['best','random'], # Forma de divisão dos dados
    'arvore__criterion': ['gini', 'entropy'] # O critério a ser utilizado é o padrão "gini", mas vai testar também utilizando a "entropy"
}

#grid_scores = ['accuracy','precision']

#Teste
# parameters_LR = {
#     "arvore__C": [0.001, 0.01, 0.1, 1.],
#     "arvore__penalty": ["l1", "l2"]
# }

#Teste
# def set_seed(seed_value):
#     random.seed(seed_value)
#     np.random.seed(seed_value)
#     os.environ["PYTHONHASHSEED"] = str(seed_value)
    
# #Teste
# SEED = 42
# set_seed(SEED)

#Teste
# model_logistic_regression = LogisticRegression(
#     random_state=SEED,
#     class_weight="balanced",
#     solver="liblinear",
# )

# # Teste
# model2_logistic_regression = GridSearchCV(
#     model_logistic_regression, 
#     parameters, 
#     cv=5,
#     scoring='accuracy',
# )

#modelo = GridSearchCV(pipetotal, param_grid=parametros, scoring=grid_scores, refit='precision', n_jobs=-1)
modelo = GridSearchCV(pipetotal, param_grid=parametros, scoring='accuracy', n_jobs=-1)

# Teste com o logisticRegression
# modelo = GridSearchCV(pipetotal, param_grid=parameters_LR, cv=5, scoring='accuracy')
#scores = cross_validate(modelo, X_train, y_train)
#scores, np.mean(scores['test_score'])

modelo.fit(X_train, y_train) # Treina com a base de treino
ypred = modelo.predict(X_test) # E faz a predição com a base de test

id_pass = X_test['PassengerId'].astype('int64') # Separa os id's dos passageiros
sobreviveu = pd.DataFrame(ypred,columns=['Survived']).astype('int64') # Separa a coluna que indica se o passageiro realmente sobreviveu
resultado = pd.concat([id_pass, sobreviveu], axis=1) # Concatena as duas colunas

np.savetxt('results_wekler.csv',resultado, delimiter=',', fmt='%i', header='PassengerId,Survived', comments="")



###########Primeiro Teste Com DecisionTree
# ({'fit_time': array([222.72679496, 219.34800124, 213.90005279, 220.2099483 ,
#          227.4420526 ]),
#   'score_time': array([0.01006532, 0.00800753, 0.0079987 , 0.00799918, 0.00794864]),
#   'test_score': array([0.78461538, 0.86      , 0.79661017, 0.86363636, 0.89130435])},
#  0.839233253113872)

###########Segundo Teste com LogisticRegression
# ({'fit_time': array([1.1089747 , 1.06800556, 1.07797122, 1.07693887, 1.09094667]),
#   'score_time': array([0.00702548, 0.00600004, 0.00605154, 0.00705194, 0.00699973]),
#   'test_score': array([0.78212291, 0.76404494, 0.79213483, 0.75842697, 0.79775281])},
#  0.7788964911179461)

###########Terceiro Teste Com DecisionTree
# ({'fit_time': array([606.93179226, 590.30258608, 590.89028478, 615.73404241,
#          765.35161567]),
#   'score_time': array([0.00699878, 0.00699425, 0.00697231, 0.00799799, 0.00794983]),
#   'test_score': array([0.79329609, 0.82022472, 0.80898876, 0.78651685, 0.83146067])},
#  0.808097420124286)

